In [6]:
import torch as th
import torch_geometric as tg
from torch import nn
from torch_geometric.nn import GATConv
from torch_geometric.data import Data, DataLoader
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import torch.optim as optim
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
import csv
from node2vec import Node2Vec


In [7]:
class DataProcessor:
    def __init__(self, 
                 dataset, 
                 data_path, 
                 k=5, 
                 gene_embedding_dim=64, 
                 string_file="string_interactions.tsv", 
                 string_query=False, 
                 output_hvg="highly_variable_genes.txt"):
        """
        Initialize the data processor with the given parameters.

        Parameters
        ----------
        dataset : str
            Name of the dataset (e.g., 'Biase', 'Darmanis', etc.)
        data_path : str
            Path to the input data file.
        k : int
            Number of nearest neighbors for adjacency construction.
        gene_embedding_dim : int
            Dimension of the gene embeddings (e.g., 64).
        string_file : str
            Path to the STRING interactions file (TSV).
        string_query : bool
            If True, attempt to query the STRING database for interactions.
        output_hvg : str
            File to write HVGs.
        """
        self.dataset = dataset
        self.data_path = data_path
        self.k = k
        self.gene_embedding_dim = gene_embedding_dim
        self.string_file = string_file
        self.string_query = string_query
        self.output_hvg = output_hvg

    def load_dataset(self, dataset_name, data_path):
        """
        Load the dataset specified by dataset_name.
        Implement your own logic here depending on how your datasets are stored.
        """
        # Placeholder: For now assume a text file containing genes x cells
        print(f"this is  {data_path}")
        adata = sc.read_text(data_path)
        
        return adata

    def preprocess_data(self, adata):
        """
        Preprocess the data:
        - Transpose if necessary
        - PCA
        - Filter genes/cells
        - Compute QC metrics, etc.
        """
        adata = adata.T
        adata.var["mt"] = adata.var_names.str.startswith("MT-")
        adata.var["ribo"] = adata.var_names.str.startswith(("RPS","RPL"))
        adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))

        sc.pp.calculate_qc_metrics(
            adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
        )

        # Basic filtering
        sc.pp.filter_cells(adata, min_genes=3)

        # PCA and neighbors
        sc.pp.pca(adata)
        sc.pp.neighbors(adata)

        # Normalize and log transform
        sc.pp.normalize_total(adata, target_sum=1e4)
        sc.pp.log1p(adata)

        # Find HVGs
        sc.pp.highly_variable_genes(adata, n_top_genes=2000)
        return adata

    def build_adjacency(self, adata, k):
        """
        Build an adjacency (KNN) graph for cells.
        """
        distances = adata.obsp["distances"]
        distances_csr = distances.tocsr()
        N = adata.shape[0]
        adj_list = []
        for i in range(N):
            row_indices = distances_csr[i].indices
            row_data = distances_csr[i].data
            # Pick top k neighbors
            k_neighbors = row_indices[row_data.argsort()[:k]]
            adj_list.append(k_neighbors)       
        A = th.zeros(size=(N, N))
        adj_list = np.array(adj_list)
        for i in range(adj_list.shape[0]):
            for j in range(k):
                A[i, adj_list[i][j]] = 1

        edge_list = th.zeros(size=(2, N*k))
        for i in range(N):
            for j in range(k):
                edge_list[0, i*k+j] = i
                edge_list[1, i*k+j] = adj_list[i][j]        
        return A, adj_list,edge_list

    def write_hvg_list(self, adata, output_file):
        hvgs = adata.var_names[adata.var["highly_variable"]]
        with open(output_file, "w") as f:
            f.write(",".join(hvgs))
        return hvgs

    def query_string_db(self, hvgs, output_file):
        """
        Query the STRING database for interactions among the given list of HVGs.
        This is pseudo-code. Implement it based on the STRING API.
        """
        # For now, assume string_interactions.tsv is already available.
        # In practice, you'd implement an API call here.
        pass

    def build_gene_graph_and_embed(self, string_file, emb_dim):
        """
        Build the gene graph from string_interactions.tsv and run node2vec.
        """
        edge_list = pd.DataFrame(csv.reader(open(string_file), delimiter="\t"))
        edge_list.columns = edge_list.iloc[0]
        edge_list = edge_list.drop(0)

        gene_graph = nx.Graph()
        gene_graph.add_edges_from(edge_list[["node1_string_id","node2_string_id"]].values)

        node2vec_model = Node2Vec(gene_graph, dimensions=emb_dim, walk_length=30, num_walks=200, workers=4)
        model = node2vec_model.fit(window=10, min_count=1, batch_words=4)
        node_embeddings = model.wv.vectors
        return node_embeddings

    def get_data(self):
        """
        Run the entire pipeline:
        - Load dataset
        - Preprocess
        - Build adjacency
        - Write HVG list
        - Optionally query STRING
        - Build gene embeddings
        Return adata, adjacency matrix A, and node_embeddings.
        """
        # 1. Load dataset
        adata = self.load_dataset(self.dataset, self.data_path)

        # 2. Preprocess
        adata = self.preprocess_data(adata)

        # 3. Build adjacency
        A, adj_list,edge_list = self.build_adjacency(adata, self.k)

        # 4. Write HVG list
        hvgs = self.write_hvg_list(adata, self.output_hvg)

        # 5. Optionally query STRING DB
        if self.string_query:
            self.query_string_db(hvgs, self.string_file)

        # 6. Build gene graph and node embeddings
        node_embeddings = self.build_gene_graph_and_embed(self.string_file, self.gene_embedding_dim)
        return adata, A, node_embeddings,edge_list


processor = DataProcessor(
    dataset="Biase",
    data_path="./tests/datastuff/GSE57249_fpkm.txt",
    k=5,
    gene_embedding_dim=64,
    string_file="./tests/datastuff/string_interactions.tsv",
    string_query=False,  # Set to True if you implement the query function
    output_hvg="./tests/datastuff/highly_variable_genes.txt"
)

In [8]:

class DeepEmbeddedClustering(nn.Module):
    def __init__(self, input_dim, hidden_dims, n_clusters, alpha=1.0):
        """
        Deep Embedded Clustering Model
        
        Args:
        - input_dim (int): Dimension of input features
        - hidden_dims (list): List of hidden layer dimensions
        - n_clusters (int): Number of clusters
        - alpha (float): Hyperparameter for soft assignment
        """
        super(DeepEmbeddedClustering, self).__init__()
        
        # Encoder layers
        encoder_layers = []
        decoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.append(GATConv(prev_dim, hidden_dim))
            decoder_layers.append(GATConv(hidden_dim, prev_dim))
            prev_dim = hidden_dim
        
        decoder_layers = decoder_layers[::-1]
        self.encoder = encoder_layers
        self.decoder = decoder_layers
        
        # Clustering layer
        self.clustering_layer = nn.Linear(hidden_dims[-1], n_clusters, bias=False)
        
        # Hyperparameters
        self.alpha = alpha
        self.n_clusters = n_clusters

    def encode(self, x, graphedge_list):
        """Forward pass through the encoder"""
        for layer in self.encoder:
            x = layer(x, graphedge_list)
        return x
    def decode(self, x, graphedge_list):
        """Forward pass through the decoder"""
        for layer in self.decoder:
            x = layer(x, graphedge_list)
        return x 
    def forward(self, x,graph_edge_list):
        """Forward pass through the network"""
        z = self.encode(x,graph_edge_list)
        q = self._soft_assignment(z)
        return z, q
    def _soft_assignment(self, z):
        """Compute soft assignment probabilities"""
        # Compute similarity between embedded points and cluster centers
        weights = self.clustering_layer.weight
        q = 1.0 / ((1.0 + th.sum((z.unsqueeze(1) - weights)**2, dim=2) / self.alpha) + 1e-8) ** ((self.alpha + 1.0) / 2.0)
        q = q / th.sum(q, dim=1, keepdim=True)
        return q
    
    def target_distribution(self, q):
        """
        Compute target distribution (sharpened version of q)
        
        Args:
        - q (th.Tensor): Soft assignment probabilities
        
        Returns:
        - p (th.Tensor): Sharpened target distribution
        """
        p = q**2 / th.sum(q, dim=0)
        p = p / th.sum(p, dim=1, keepdim=True)
        return p.detach()

In [9]:

def pretrain_autoencoder(model, data,adj_list, epochs=50, lr=1e-3):
    """
    Pretrain autoencoder for initial feature extraction
    
    Args:
    - model (DeepEmbeddedClustering): DEC model
    - data (th.Tensor): Input data
    - epochs (int): Number of pretraining epochs
    - lr (float): Learning rate
    
    Returns:
    - Pretrained model
    """
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        z = model.encode(data,adj_list)
        reconstruction = model.decode(z,adj_list)
        loss = criterion(reconstruction, data)
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print(f'Pretraining Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}')
    
    return model

def dec_loss(p, q):
    """
    Compute DEC loss (KL divergence between p and q)
    
    Args:
    - p (th.Tensor): Target distribution
    - q (th.Tensor): Soft assignment probabilities
    
    Returns:
    - loss (th.Tensor): KL divergence loss
    """
    return th.mean(th.sum(p * th.log(p / q), dim=1))

def train_dec(model,edge_list, data, epochs=100, lr=1e-3):
    """
    Train Deep Embedded Clustering model
    
    Args:
    - model (DeepEmbeddedClustering): DEC model
    - data (th.Tensor): Input data
    - epochs (int): Number of training epochs
    - lr (float): Learning rate
    
    Returns:
    - Trained model and cluster assignments
    """
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Initialize cluster centers using KMeans
    with th.no_grad():
        z = model.encode(data,edge_list)
        kmeans = KMeans(n_clusters=model.n_clusters, n_init=20)
        cluster_labels = kmeans.fit_predict(z.numpy())
        
        # Set initial cluster centers
        model.clustering_layer.weight.copy_(
            th.from_numpy(kmeans.cluster_centers_).float()
        )
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        
        # Forward pass
        z, q = model(data,edge_list)
        
        # Compute target distribution
        p = model.target_distribution(q)
        
        # Compute loss
        loss = dec_loss(p, q)
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print(f'DEC Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}')
    
    # Get final cluster assignments
    _, q = model(data,edge_list)
    cluster_assignments = th.argmax(q, dim=1)
    
    return model, cluster_assignments



In [10]:
# Simulated data
processor = DataProcessor(
    dataset="Biase",
    data_path="./datasets/Biase/GSE57249_fpkm.txt",
    k=5,
    gene_embedding_dim=64,
    string_file="./datasets/Biase/string_interactions.tsv",
    string_query=False,  # Set to True if you implement the query function
    output_hvg="./datasets/Biase/highly_variable_genes.txt"
)
adata, A, node_embeddings,edge_list = processor.get_data()
# so our input is the sc matrix of only the hvg genes
hvg = adata.var['highly_variable']
adata_hvg = adata[:, hvg].X
data = th.tensor(adata_hvg)


this is  ./datasets/Biase/GSE57249_fpkm.txt


c:\Users\saira\anaconda3\envs\Pytorch\Lib\site-packages\scanpy\preprocessing\_highly_variable_genes.py:305: RuntimeWarning: invalid value encountered in log
  dispersion = np.log(dispersion)
Computing transition probabilities: 100%|██████████| 647/647 [00:00<00:00, 5576.08it/s]


In [31]:
input_dim = data.shape[1]
hidden_dims = [256, 128, 64]
n_clusters = 3
# Initialize model

In [114]:
model = DeepEmbeddedClustering(input_dim, hidden_dims, n_clusters)
edge_list = edge_list.int()
pretrain_autoencoder(model,data,edge_list, epochs=200, lr=2e-4)

# Train model
trained_model, cluster_labels = train_dec(model,edge_list ,data,epochs=5000,lr=5e-4)

print("Clustering complete. Cluster labels:", cluster_labels)

# Compare with sklearn clustering
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data.numpy())

# Fit KMeans from sklearn
kmeans_sklearn = KMeans(n_clusters=n_clusters, n_init=20)
cluster_labels_sklearn = kmeans_sklearn.fit_predict(data_scaled)

# Print comparison
print("Cluster labels from DEC model:", cluster_labels.numpy())
print("Cluster labels from sklearn KMeans:", cluster_labels_sklearn)

# Compute ARI and NMI


Pretraining Epoch [0/200], Loss: 0.3895
Pretraining Epoch [10/200], Loss: 0.3895
Pretraining Epoch [20/200], Loss: 0.3895
Pretraining Epoch [30/200], Loss: 0.3895
Pretraining Epoch [40/200], Loss: 0.3895
Pretraining Epoch [50/200], Loss: 0.3895
Pretraining Epoch [60/200], Loss: 0.3895
Pretraining Epoch [70/200], Loss: 0.3895
Pretraining Epoch [80/200], Loss: 0.3895
Pretraining Epoch [90/200], Loss: 0.3895
Pretraining Epoch [100/200], Loss: 0.3895
Pretraining Epoch [110/200], Loss: 0.3895
Pretraining Epoch [120/200], Loss: 0.3895
Pretraining Epoch [130/200], Loss: 0.3895
Pretraining Epoch [140/200], Loss: 0.3895
Pretraining Epoch [150/200], Loss: 0.3895
Pretraining Epoch [160/200], Loss: 0.3895
Pretraining Epoch [170/200], Loss: 0.3895
Pretraining Epoch [180/200], Loss: 0.3895
Pretraining Epoch [190/200], Loss: 0.3895
DEC Epoch [0/5000], Loss: 0.0822
DEC Epoch [10/5000], Loss: 0.0820
DEC Epoch [20/5000], Loss: 0.0818
DEC Epoch [30/5000], Loss: 0.0816
DEC Epoch [40/5000], Loss: 0.0814
DE

In [115]:
celltypes = pd.read_csv("./datasets/Biase/subtype.ann",delimiter="\t",header=None)
celltypes.columns = ["cell","type"]

In [116]:
celltypes["type"].values

array(['label', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2',
       '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2'],
      dtype=object)

In [117]:
indices = []
for cell in celltypes["cell"]:
    if cell in adata.obs_names:
        index = adata.obs_names.get_loc(cell)
        print(f"Index of cell {cell}: {index}")
        indices.append(index)
    else:
        print(f"Cell {cell} not found in adata.obs_names")


Cell cell not found in adata.obs_names
Index of cell GSM1377859: 0
Index of cell GSM1377860: 1
Index of cell GSM1377861: 2
Index of cell GSM1377862: 3
Index of cell GSM1377863: 4
Index of cell GSM1377864: 5
Index of cell GSM1377865: 6
Index of cell GSM1377866: 7
Index of cell GSM1377867: 8
Index of cell GSM1377868: 9
Index of cell GSM1377869: 10
Index of cell GSM1377870: 11
Index of cell GSM1377871: 12
Index of cell GSM1377872: 13
Index of cell GSM1377873: 14
Index of cell GSM1377874: 15
Index of cell GSM1377875: 16
Index of cell GSM1377876: 17
Index of cell GSM1377877: 18
Index of cell GSM1377878: 19
Index of cell GSM1377879: 20
Index of cell GSM1377880: 21
Index of cell GSM1377881: 22
Index of cell GSM1377882: 23
Index of cell GSM1377883: 24
Index of cell GSM1377884: 25
Index of cell GSM1377885: 26
Index of cell GSM1377886: 27
Index of cell GSM1377887: 28
Index of cell GSM1377888: 29
Index of cell GSM1377889: 30
Index of cell GSM1377890: 31
Index of cell GSM1377891: 32
Index of cell 

In [118]:
cluster_labels[:49]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2,
        0])

In [119]:
celltypes.values[:,1][1:]

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2',
       '2', '2', '2', '2', '2', '2', '2', '2', '2', '2'], dtype=object)

In [120]:


ari = adjusted_rand_score(cluster_labels[:49],celltypes.values[:,1][1:])
nmi = normalized_mutual_info_score(cluster_labels[:49],celltypes.values[:,1][1:])

print(f"Adjusted Rand Index (ARI): {ari:.4f}")
print(f"Normalized Mutual Information (NMI): {nmi:.4f}")

print("Sklearn ARI and NMI:")
ari = adjusted_rand_score(cluster_labels_sklearn[:49],celltypes.values[:,1][1:])
nmi = normalized_mutual_info_score(cluster_labels_sklearn[:49],celltypes.values[:,1][1:])

print(f"Adjusted Rand Index (ARI): {ari:.4f}")
print(f"Normalized Mutual Information (NMI): {nmi:.4f}")



Adjusted Rand Index (ARI): 0.3588
Normalized Mutual Information (NMI): 0.5534
Sklearn ARI and NMI:
Adjusted Rand Index (ARI): 0.7114
Normalized Mutual Information (NMI): 0.7754
